# Part-II: VacationPy

In [1]:
# Dependencies and Setup
from IPython.display import display
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import 'API' and 'G' Keys
from config import gkey
from config import (api_key)

# Configure 'G' Key
gmaps.configure(api_key = gkey)

### Store Part-I results into DataFrame
* Load the CSV that was exported in Part-I, to a DataFrame

In [2]:
# Dependencies and Setup
pro_dir = ''

cities_path = "../selected_cities/selected_cities.csv"

# Read the 'Cities' data that we just stored in a CSV file
cities_data = pd.read_csv(pro_dir + cities_path)

locations = cities_data[['Latitude', 'Longitude']]
hum_weights = cities_data['Humidity (%)']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights = hum_weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
cities_utemp_limit = cities_data[cities_data['Max. Temp (F)'] < 80]
cities_ultemp_limit = cities_utemp_limit[cities_utemp_limit['Max. Temp (F)'] > 70]
cities_ftemp_fws = cities_ultemp_limit[cities_ultemp_limit['Wind Speed (mph)'] < 10]
cities_filtered = cities_ftemp_fws[cities_ftemp_fws['Cloudiness (%)'] == 0]
display(cities_filtered)
targ_rad = 5000

,City Name,Latitude,Longitude,Temperature (F),Max. Temp (F),Min. Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country Code,Date & Time
34,doha,25.2867,51.5333,71.15,71.60,69.80,43,0,5.75,QA,14-01-2021 01:01:39
68,elat,29.5581,34.9482,75.20,75.20,75.20,57,0,8.05,IL,14-01-2021 01:02:20
121,chuy,-33.6971,-53.4616,72.27,72.27,72.27,46,0,7.96,UY,14-01-2021 01:06:32
183,mar del plata,-38.0023,-57.5575,72.99,75.20,72.00,33,0,8.05,AR,14-01-2021 01:07:51
239,nelson bay,-32.7167,152.1500,70.65,72.00,69.01,83,0,8.05,AU,14-01-2021 01:09:07
324,san juan,-31.5375,-68.5364,79.59,79.59,79.59,37,0,5.48,AR,14-01-2021 01:06:15
349,bundaberg,-24.8500,152.3500,70.43,71.01,69.80,83,0,5.75,AU,14-01-2021 01:11:42
354,vallenar,-28.5708,-70.7581,74.91,74.91,74.91,39,0,4.59,CL,14-01-2021 01:11:48
363,necochea,-38.5473,-58.7368,71.40,71.40,71.40,29,0,6.22,AR,14-01-2021 01:11:59
409,catuday,16.2923,119.8062,76.01,76.01,76.01,76,0,9.22,PH,14-01-2021 01:13:04


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [4]:
final_rest_data = {'Latitude':[],
                   'Longitude':[],
                   'Restaurant (Eatery) Name':[],
                   'City':[],
                   'Country Code':[]
                  }

for index, row in cities_filtered.iterrows():
    coordinates = str(row['Latitude']) + ', ' + str(row['Longitude'])
    params = {"location": coordinates,
              "type": "restaurant",
              'radius': targ_rad,
              "key": gkey
             }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params = params)
    restaurants_places = response.json()
    
    try:
        rest_data = restaurants_places['results'][0]
        rs_nm = rest_data['name']
        final_rest_data['Restaurant (Eatery) Name'].append(rest_data['name'])
        final_rest_data['Latitude'].append(restaurants_places['results'][0]['geometry']['location']['lat'])
        final_rest_data['Longitude'].append(restaurants_places['results'][0]['geometry']['location']['lng'])
        final_rest_data['City'].append(row['City Name'])
        final_rest_data['Country Code'].append(row['Country Code'])
    except:
        aaa = 0
hotel_df = pd.DataFrame(final_rest_data)
display(hotel_df)

,Latitude,Longitude,Restaurant (Eatery) Name,City,Country Code
0,25.289950,51.542760,Best Western Plus Doha,doha,QA
1,29.548334,34.966198,לורנס Lawrence,elat,IL
2,-33.692020,-53.454269,La Fortaleza,chuy,UY
3,-38.015371,-57.541319,Hotel Sainte Jeanne,mar del plata,AR
4,-32.720833,152.143611,Seabreeze Hotel,nelson bay,AU
5,-31.538557,-68.554266,Del Bono Park Hotel Spa & Casino,san juan,AR
6,-24.872265,152.333805,Takalvan Motel,bundaberg,AU
7,-28.574664,-70.758609,Hostal Camino del Rey,vallenar,CL
8,-38.577542,-58.728594,Sabores,necochea,AR
9,16.301765,119.781714,Ron-Gel Resto Grill,catuday,PH


In [5]:
# NOTE: Do not change any of the code in this cell
# Using the template, add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Restaurant (Eatery) Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: Be sure to update with your DataFrame name

hotel_info_b = ['Name \n' + row['Restaurant (Eatery) Name'] + '\n City \n' + row['City'] + '\n Country \n' + row['Country Code'] for index, row in hotel_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
marker_layer = gmaps.marker_layer(locations, hover_text = hotel_info_b)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))